# Preparation

Make sure to download the corresponding medical ontologies to build the term dictionaries. For each, look for the 2017 version.
 - Treatments: [OPS](https://www.bfarm.de/DE/Kodiersysteme/Services/Downloads/_node.html).
 - Medications: [ATC](https://www.wido.de/publikationen-produkte/arzneimittel-klassifikation/).
 - Diagnosis: [ICD10GM](https://www.bfarm.de/DE/Kodiersysteme/Services/Downloads/_node.html). 
 
In the config file for BRONCO `conf/bronco.yaml`, modify the paths so they point the extracted folders. We assume they are located in `xmen/local_files`. Otherwise, change the path here and correct accordingly the terminal commands below:

We can already use xMEN to prepare the term dictionaries. <u>This step only has to be performed the first time. </u> 

In your terminal, navigate to the xMEN root folder and run:
 - `xmen dict examples/conf/bronco.yaml --code examples/dicts/atc2017_de.py --key atc`
 - `xmen dict examples/conf/bronco.yaml --code examples/dicts/ops2017.py --key ops`
 - `xmen dict examples/conf/bronco.yaml --code examples/dicts/icd10gm2017.py --key icd10gm`
 
Now use such dictionaries to build the indexes:
 - `xmen index examples/conf/bronco.yaml --dict ~/.cache/xmen/atc/atc.jsonl --output ~/.cache/xmen/atc/ --all`
 - `xmen index examples/conf/bronco.yaml --dict ~/.cache/xmen/ops/ops.jsonl ~/.cache/xmen/ops/ --all`
 - `xmen index examples/conf/bronco.yaml --dict ~/.cache/xmen/icd10gm/icd10gm.jsonl ~/.cache/xmen/icd10gm/ --all`
 
Now we can load the BRONCO150 dataset using BigBIO:

In [2]:
import datasets

path_to_data = r"../local_files/BRONCO150" # paste here the path to the local data

bronco = datasets.load_dataset(path = "bigbio/bronco", 
                               name = "bronco_bigbio_kb", 
                               data_dir=path_to_data)

bronco

Found cached dataset bronco (/home/Florian.Borchert/.cache/huggingface/datasets/bigbio___bronco/bronco_bigbio_kb-fd41eed48d3255b6/1.0.0/cab8fc4a62807688cb5b36df7a24eb7f364314862c4196f6ff2db3813f2fe68b)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'document_id', 'passages', 'entities', 'events', 'coreferences', 'relations'],
        num_rows: 5
    })
})

Finally, we have to choose the semantic class we want to work on and reestructure the dataset in 5 folds for cross-validation, as originally intended.

In [3]:
label = "MEDICATION" # Choose here TREATMENT, MEDICATION or DIAGNOSIS

In [4]:
label2dict = {
    "TREATMENT": "ops",
    "MEDICATION": "atc",
    "DIAGNOSIS": "icd10gm"
}

def filter_entities(bigbio_entities, valid_entities):
    filtered_entities = []
    for ent in bigbio_entities:
        if ent['type'] in valid_entities:
            filtered_entities.append(ent)
    return filtered_entities

ds = bronco.map(lambda row: {'entities': filter_entities(row['entities'], [label])})

Loading cached processed dataset at /home/Florian.Borchert/.cache/huggingface/datasets/bigbio___bronco/bronco_bigbio_kb-fd41eed48d3255b6/1.0.0/cab8fc4a62807688cb5b36df7a24eb7f364314862c4196f6ff2db3813f2fe68b/cache-44459d7791d9d192.arrow


In [5]:
from datasets import DatasetDict

ground_truth = DatasetDict()
for k in range(5):
    ground_truth[f"k{k+1}"] = ds["train"].select([k])
    
ds = ground_truth
ds

DatasetDict({
    k1: Dataset({
        features: ['id', 'document_id', 'passages', 'entities', 'events', 'coreferences', 'relations'],
        num_rows: 1
    })
    k2: Dataset({
        features: ['id', 'document_id', 'passages', 'entities', 'events', 'coreferences', 'relations'],
        num_rows: 1
    })
    k3: Dataset({
        features: ['id', 'document_id', 'passages', 'entities', 'events', 'coreferences', 'relations'],
        num_rows: 1
    })
    k4: Dataset({
        features: ['id', 'document_id', 'passages', 'entities', 'events', 'coreferences', 'relations'],
        num_rows: 1
    })
    k5: Dataset({
        features: ['id', 'document_id', 'passages', 'entities', 'events', 'coreferences', 'relations'],
        num_rows: 1
    })
})

# Run Candidate Generator
We will use the default `EnsembleLinker`:

In [6]:
from xmen.linkers import default_ensemble
from xmen.evaluation import evaluate_at_k
import os
from pathlib import Path

index_base_path = Path(f"{os.path.expanduser('~')}/.cache/xmen/{label2dict[label]}/index")
linker = default_ensemble(index_base_path)

[10/18/23 19:58:31] INFO     Loading hierarchical faiss index                                ]8;id=682292;file:///mnt/nfs/home/Florian.Borchert/workspace/xmen/xmen/linkers/sap_bert_linker.py\sap_bert_linker.py]8;;\:]8;id=417359;file:///mnt/nfs/home/Florian.Borchert/workspace/xmen/xmen/linkers/sap_bert_linker.py#153\153]8;;\

                    INFO     Loading index from                                                 ]8;id=288595;file:///mnt/nfs/home/Florian.Borchert/workspace/xmen/xmen/linkers/faiss_indexer.py\faiss_indexer.py]8;;\:]8;id=41492;file:///mnt/nfs/home/Florian.Borchert/workspace/xmen/xmen/linkers/faiss_indexer.py#64\64]8;;\
                             /home/Florian.Borchert/.cache/xmen/atc/index/sapbert/embed_faiss_h                    
                             ier.pickle                                                                            

[10/18/23 19:58:32] INFO     Loaded index of type <class 'faiss.swigfaiss.IndexHNSWFlat'> and   ]8;id=346283;file:///mnt/nfs/home/Florian.Borchert/workspace/xmen/xmen/linkers/faiss_indexer.py\faiss_indexer.py]8;;\:]8;id=436706;file:///mnt/nfs/home/Florian.Borchert/workspace/xmen/xmen/linkers/faiss_indexer.py#66\66]8;;\
                             size 477959                                                                           

In [7]:
candidates = linker.predict_batch(ds)

# Recall for different numbers of candidates (k)
_ = evaluate_at_k(ds['k5'], candidates['k5'])

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Recall@1 0.44375
Recall@2 0.525
Recall@4 0.64375
Recall@8 0.7375
Recall@16 0.771875
Recall@32 0.8125
Recall@64 0.81875


# Train Cross-encoder
We use a cross-encoder to embed the mention with their context together with all potential candidates. This way, we can learn the best ranking of candidates from the training data.

In [8]:
from xmen.reranking.cross_encoder import CrossEncoderReranker, CrossEncoderTrainingArgs
from xmen.data import filter_and_apply_threshold
from xmen import load_kb

In [9]:
K_RERANKING = 64
candidates = filter_and_apply_threshold(candidates, K_RERANKING, 0.0)
kb = load_kb(index_base_path.parent / f"{label2dict[label]}.jsonl")

cross_enc_ds = CrossEncoderReranker.prepare_data(candidates, ds, kb)
cross_enc_ds

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

...

  0%|          | 0/314 [00:00<?, ?it/s]

  0%|          | 0/320 [00:00<?, ?it/s]

  0%|          | 0/320 [00:00<?, ?it/s]

  0%|          | 0/320 [00:00<?, ?it/s]

{'k1': [325 items],
 'k2': [333 items],
 'k3': [338 items],
 'k4': [314 items],
 'k5': [320 items]}

Now we set the training arguments, train-eval splits and fit the model. Depending on the number of epochs, training can take several hours.

In [10]:
# Choose train and evaluation folds
train_folds = ["k1", "k2", "k3"]
val_fold = "k4"
test_fold = "k5"

train = sum([cross_enc_ds[k].dataset for k in train_folds],[])
val = cross_enc_ds[val_fold].dataset

In [11]:
train_args = CrossEncoderTrainingArgs(num_train_epochs = 5)

rr = CrossEncoderReranker()
output_dir = f'../outputs/{label2dict[label]}_index/cross_encoder_training/'

In [12]:
rr.fit(
    train_dataset = train,
    val_dataset = val,
    output_dir= output_dir,
    training_args = train_args,
    show_progress_bar = False
)

model_name := bert-base-multilingual-cased
num_train_epochs := 5
fp16 := True
label_smoothing := False
rank_regularization := 1.0
train_layers := None
softmax_loss := True
random_seed := 42
learning_rate := 2e-05


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual

2023-10-18 20:19:11 - EntityLinkingEvaluator: Evaluating the model on eval dataset after epoch 0:
2023-10-18 20:22:01 - Accuracy: 0.5191082802547771
2023-10-18 20:22:01 - Accuracy @ 5: 0.7643312101910829
2023-10-18 20:22:01 - Accuracy @ 64: 1.0
2023-10-18 20:22:01 - Baseline Accuracy: 0.4394904458598726
2023-10-18 20:22:01 - Save model to ../outputs/atc_index/cross_encoder_training/
2023-10-18 20:37:41 - EntityLinkingEvaluator: Evaluating the model on eval dataset after epoch 1:
2023-10-18 20:40:31 - Accuracy: 0.6019108280254777
2023-10-18 20:40:31 - Accuracy @ 5: 0.8757961783439491
2023-10-18 20:40:31 - Accuracy @ 64: 1.0
2023-10-18 20:40:31 - Baseline Accuracy: 0.4394904458598726
2023-10-18 20:40:31 - Save model to ../outputs/atc_index/cross_encoder_training/
2023-10-18 20:56:12 - EntityLinkingEvaluator: Evaluating the model on eval dataset after epoch 2:
2023-10-18 20:59:02 - Accuracy: 0.60828025477707
2023-10-18 20:59:02 - Accuracy @ 5: 0.8949044585987261
2023-10-18 20:59:02 - Accu

# Evaluate Cross-encoder
Now we can take our trained model and test it on data outside of training.

In [13]:
rr = CrossEncoderReranker.load(output_dir, device=0)

2023-10-18 21:36:07 - Use pytorch device: cuda


In [14]:
cross_enc_pred_val = rr.rerank_batch(candidates[val_fold], cross_enc_ds[val_fold])
_ = evaluate_at_k(ds[val_fold], cross_enc_pred_val)

Batches:   0%|          | 0/314 [00:00<?, ?it/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Recall@1 0.7197452229299363
Recall@2 0.7547770700636943
Recall@4 0.7770700636942676
Recall@8 0.7866242038216561
Recall@16 0.7993630573248408
Recall@32 0.802547770700637
Recall@64 0.802547770700637


In [15]:
cross_enc_pred_test = rr.rerank_batch(candidates[test_fold], cross_enc_ds[test_fold])
_ = evaluate_at_k(ds[test_fold], cross_enc_pred_test)

Batches:   0%|          | 0/320 [00:00<?, ?it/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Recall@1 0.675
Recall@2 0.725
Recall@4 0.734375
Recall@8 0.78125
Recall@16 0.790625
Recall@32 0.8
Recall@64 0.80625
